In [1]:
from ner.utils import download_untar
import numpy as np
from sklearn.model_selection import train_test_split
from ner.network import NER
import os
from tqdm import tqdm_notebook
import re
from ner.utils import tokenize, lemmatize
import json

from os.path import join

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset_dict = dict()

In [3]:
# with open(join('.', 'NER_concept.txt'), encoding='utf-8') as f:
#     xy_list = list()
#     tokens = list()
#     tags = list()
#     val = list()
#     for line in tqdm_notebook(f):
#         items = line.split()
#         if len(items) > 1 and '-DOCSTART-' not in items[0]:
#             token, tag = items
#             token_ = tokenize(token)
#             if len(token_) > 0:
#                 token = token_[0]
#             if token[0].isdigit():
#                 tokens.append('#')
#             else:
#                 tokens.append(token)
#             tags.append(tag)
#         elif len(tokens) > 0:
#             xy_list.append((tokens, tags,))
#             tokens = list()
#             tags = list()


In [3]:
xy_list = np.load('NER_set_simple.npy')

In [4]:
xy_list = np.append(xy_list, np.load('NER_set_conll_2003.npy'), axis=0)

In [5]:
dataset_dict['train'], ost = train_test_split(xy_list,test_size=0.3)

In [6]:
dataset_dict['valid'], dataset_dict['test'] = train_test_split(ost,test_size=0.5)

In [8]:
#np.save('NER_set_simple.npy', xy_list)

In [9]:
#np.save('NER_set_conll_2003.npy', xy_list)

In [7]:
from ner.corpus import Corpus
corp = Corpus(dataset_dict, embeddings_file_path='model185.txt')

Loading embeddins...


In [8]:
with open('my_model/params.json') as f:
    network_params = json.load(f)

In [9]:
net = NER(corp, verbouse=False, pretrained_model_filepath='my_model/my_model', **network_params)

INFO:tensorflow:Restoring parameters from my_model/my_model


In [34]:
# net = NER(corp, token_embeddings_dim=300, use_crf=True, char_embeddings_dim=25,
# concat_embeddings=True, use_char_embeddins=True)
# learning_params = {'dropout_rate': 0.5,
#                    'epochs': 20,
#                    'learning_rate': 0.005,
#                    'batch_size': 1}

# results = net.fit(**learning_params)

Number of parameters: 
Embeddings 5826150
ConvNet 338688
Classifier 1542
transitions:0 36
Total number of parameters equal 6166416


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 0
Eval on valid:
processed 11361 tokens with 297 phrases; found: 277 phrases; correct: 173.

precision:  62.45%; recall:  58.25%; FB1:  60.28


Epoch 1
Eval on valid:
processed 11361 tokens with 297 phrases; found: 224 phrases; correct: 177.

precision:  79.02%; recall:  59.60%; FB1:  67.95


Epoch 2
Eval on valid:
processed 11361 tokens with 297 phrases; found: 239 phrases; correct: 185.

precision:  77.41%; recall:  62.29%; FB1:  69.03


Epoch 3
Eval on valid:
processed 11361 tokens with 297 phrases; found: 329 phrases; correct: 158.

precision:  48.02%; recall:  53.20%; FB1:  50.48


Epoch 4
Eval on valid:
processed 11361 tokens with 297 phrases; found: 231 phrases; correct: 161.

precision:  69.70%; recall:  54.21%; FB1:  60.98


Epoch 5
Eval on valid:
processed 11361 tokens with 297 phrases; found: 276 phrases; correct: 190.

precision:  68.84%; recall:  63.97%; FB1:  66.32


Epoch 6
Eval on valid:
processed 11361 tokens with 297 phrases; found: 319 phrases; correct: 190.

p

In [10]:
def make_predict(sentence, network):
    tokens = tokenize(sentence)
    if len(tokens) == 0:
        tokens = ['']    
    tags = network.predict_for_token_batch([tokens])[0]
    return tokens, tags

def print_predict(tokens, tags):
    NERS = []
    k = 0
    curr_ner = []
    for token, tag in zip(tokens, tags):
        if tag != 'O' and len(token) >= 2:
            curr_ner.append(token)
        else:
            if len(curr_ner) > 0:
                NERS.append(" ".join(curr_ner))
            curr_ner = []
    if len(curr_ner) > 0:
        NERS.append(" ".join(curr_ner))
    return set(NERS)

In [11]:
S = "Мария сдает работу Ивану Захарову в МФТИ"

In [12]:
x, y = make_predict(S, net)

In [13]:
x, y

(['Мария', 'сдает', 'работу', 'Ивану', 'Захарову', 'в', 'МФТИ'],
 ['B-PER', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-LOC'])

In [14]:
print_predict(x, y)

{'Ивану Захарову', 'МФТИ', 'Мария'}

In [15]:
points = ['.', '!', '&', '?', '...', ')', '(']

In [ ]:
for name in os.listdir('test_set'):
    print(name, "___________________________________________________________________________________________")
    NERS = set()
    with open('test_set/' + name) as f:
        for s in f.readlines():
            if s == "\n" or s == " " or s == "\t":
                continue
            pred=make_predict(s, net)
            NERS = NERS | print_predict(pred[0], pred[1])             
    f = open('result/' + name, mode='a')
    k = 0
    for ner in NERS:
        if ner not in points and len(ner) > 2 and not ner.isdigit(): 
            print(ner)
            k += 1
            f.write(ner + "\n")
    print(k)
    f.close()    

20130822 РГ КП Неретин.txt ___________________________________________________________________________________________
Салтыковкой
Ленинке
Гугла
Наверняка
Московский
Достоевского
Пермском хореографическом
Инфраструктура
Екатеринбурге
Строгановка
Московских
Расфасовав
Питерский
Большой театр
Интересная пропорция
Белгородской области
Новгородке
Пушкинском музее
Пентхаус
Москву
России
Второй
Кучкаров
Академия Глазунова
Профессии
Статья
Попса
Забыл
СССР
Свинарка
Рекламу Винзавода
Дифференцируем
ВВП
Петербурге
Российской
Рубенса
Московская
Марату Гельману
ДШИ
Юрий Иванович
Риме
Кемерово
Реставраторы
Хорошо
Руководством
Школа Табакова
Кажется
Репинка
Ленинкой
Абстракции
ВГИК
движимого
Сеть культовых
Центральной
ГИТИСе
Санкт
Руководитель
например
Москве
59
20130910 КП интервью Смирнов.txt ___________________________________________________________________________________________
Израиля
Ливана читал
Салман Рушди
Александр Матросов
завершить
Калягина
Плясать
Европе
Сколково
СПб
английски
Казах